# Notebook for evaluating workshop recordings

The audio of the birthday song from Hurra for deg! - Norwegian Birthday Song by 
aquaskaya at https://www.youtube.com/watch?v=7mIKZbl_RO0

Hurra for deg som fyller ditt år!
Ja, deg vil vi gratulere!
Alle i ring omkring deg vi står,
og se, nå vil vi marsjere.
Bukke nikke neie, snu oss omkring,
danse for deg med hopp og sprett og spring.
Ønske deg av hjertet alle gode ting!
Og si meg så, hva vil du mere?
Gratulere!


### Prepare notebook

In [ ]:
!git clone https://github.com/finn42/Beat_Breath_Workshop.git

Drag the qex.py, mh.py, and rp2.py modules into the main folder


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import qex
import mh
import rp2

## Import data

In [ ]:
userID = '3420755'
suffix = '-260107.CSV'
offset = 10
dataloc_url = 'https://raw.githubusercontent.com/finn42/Trimming_EQ_Recordings/refs/heads/main/Workshop_Demo/'

ECGLoc = dataloc_url + 'ECG-' + userID+ '-' + userID + '-260107.csv'
V=pd.read_csv(ECGLoc)
V['DateTime'] = pd.to_datetime(V['DateTime'])
V['Time_s'] = (V['DateTime']-V.loc[0,'DateTime']).dt.total_seconds()-offset
ECG = V.set_index('Time_s')

RRLoc = dataloc_url + 'RR-' + userID+ '-' + userID + suffix
V=pd.read_csv(RRLoc)
V['DateTime'] = pd.to_datetime(V['DateTime'])
V['Time_s'] = (V['DateTime']-V.loc[0,'DateTime']).dt.total_seconds()-offset
IBI = V.set_index('Time_s')
cIBI=mh.clean_IBI(V['Time_s'],V['Interbeat Interval (MS)'],activitytype='default')
newIBI = mh.contact_noise_cut(ECG.index,ECG['Lead 1'].values,V['Time_s'],V['Interbeat Interval (MS)'],0.3)

newV = V.loc[newIBI.index,:]
newIBI=mh.clean_IBI(newV['Time_s'],newV['Interbeat Interval (MS)'],activitytype='default')
df_Card = mh.ibi_sbeatfeats(newIBI.index,newIBI['IBI'].values)

# ACCLoc = dataloc_url + 'ACCFAST-' + userID+ '-' + userID + suffix
# ACC=pd.read_csv(ACCLoc,index_col=0)

In [ ]:
RESPLoc = dataloc_url + 'RESPACC-' + userID+ '-' + userID + suffix
V=pd.read_csv(RESPLoc)

V['DateTime'] = pd.to_datetime(V['DateTime'])
V['Time_s'] = (V['DateTime']-V.loc[0,'DateTime']).dt.total_seconds()-offset
Resp = V.set_index('Time_s').copy()
Breaths = rp2.breath_cycles(rp2.EQ_Inspiration_Extract(Resp['Breathing']),Resp['Breathing'])
Insp_seq = rp2.quickcarresp(Breaths)

In [ ]:
demofile_loc = './Beat_Breath_Workshop/demodata/' #location for colab

Events = pd.read_csv(demofile_loc+'timing/Demo_Events_Structure.csv')
Events = Events.loc[Events.Structure.str.len()>0,:].copy()
Events['Duration']=Events['TIME'].diff().shift(-1)
events_pallet = {'Seated':'blue','Standing':'green','Vocal':'yellow','Running':'red'}

# plot Cardiac activity and Respiration together

In [ ]:
fig,(axes) = plt.subplots(3,1,figsize=(12,8),sharex=True)
ax = axes[0]
df_Card[['HR1bt', 'HR10s', 'HR30s']].plot(ax = ax)
ax.grid()
for cat in events_pallet.keys(): 
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:-1,:].iterrows():
    ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['Structure']],alpha = 0.2)
ax.set(xlim = [0,165],ylim = [60,160],ylabel='Heart Rate (BPM)',xlabel = 'Time (s)')
ax.set_title('Heart Rate during demo actions') 

ax = axes[1]
ax.scatter(Insp_seq['In'],Insp_seq['In_C'],label = 'Inspiration onset')
ax.scatter(Insp_seq['Ex'],Insp_seq['Ex_C'],c='r',label = 'Expiration onset')
Resp['Breathing'].plot(ax = ax,label='Respiration wave')
ax.set(ylabel = 'Resp wave')
for i,row in Events.iloc[:-1,:].iterrows():
    ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['Structure']],alpha = 0.2)

ax = axes[2]
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'],label = ['Period_T'])
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'].rolling(7,center=True).median(),label = ['Period_T'])
ax.grid()
for cat in events_pallet.keys(): #'Vocal?',
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:-1,:].iterrows():
    ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['Structure']],alpha = 0.2)
ax.set(xlim = [0,200],ylim = [0,13],ylabel='Respiration Period (s)',xlabel = 'Time (s)')

plt.show()

Try to plot other graphs of the demo data using snippets of code from other notebooks